# Geonames recommendation system

**Project description:** The career center needs to be able to match random city names with the unified geonames for internal use. The cities in question might be located in Russia, Belarus, Armenia, Kazakhstan, Kyrgyzstan, Georgia or Serbia. The operator should receive a list of recommended names which also contains a geonameid, a region, a country and a cosine similarity score for each city name.

**Task:** To create a recommendation system that receives a random city name as a query and returns a list of tuples with recommended names in unified spelling and geonameid's for these locations.

**Data description:** datasets from download.geonames.org which contain city names, geonameid's, region and country info, etc. (admin1CodesASCII, alternateNamesV2, cities15000, countryInfo).

**Work plan:**

1 DATA PREPROCESSING

1.1 Importing necessary libraries

1.2 Loading the datasets into corresponding variables, putting them and info about them on screen

1.3 Creating a working dataset

1.4 Preprocessing the working dataset

1.5 Summary

2 APPLYING THE SENTENCE TRANSFORMER

2.1 Applying the Sentence Transformer

2.2 Summary

3 EXPANDING THE RESULTING DATASET

3.1 Expanding the resulting dataset

3.2 Creating a function for a geoname queries

3.3 Summary

4 TESTING THE SOLUTION

4.1 Testing the solution

4.2 Summary

5 CONCLUSION

## Data preprocessing

### Importing necessary libraries

In [ ]:
# !pip install SQLAlchemy
# !pip install --pre SQLAlchemy
# !pip install psycopg2

In [ ]:
# !pip install -U sentence-transformers

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

import pandas as pd
import unidecode

from sentence_transformers import SentenceTransformer, util

Creating a connection to PostgreSQL:

In [2]:
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'bathack73',
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'query': {}
}

In [3]:
engine = create_engine(URL.create(**DATABASE))

### Loading the datasets into corresponding variables, putting them and info about them on screen

Loading the dataset with administrative codes for regions as `admin_divisions`:

In [ ]:
admin_divisions = pd.read_csv(
    'C:/Users/ASUS/Documents/Projects/Geonames_matching/admin1CodesASCII.txt',
    delimiter='\t',
    encoding='utf-16',     # setting the unicode type
    header=None,
    names=[                # naming the columns
        'code',            # administrative code
        'region',          # region name
        'region_ascii',    # region ASCII name
        'geonameid_admin'  # region geoname id
    ]
)
admin_divisions.head()

,code,region,region_ascii,geonameid_admin
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131
3,AD.03,Encamp,Encamp,3040684
4,AD.02,Canillo,Canillo,3041203


In [ ]:
admin_divisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3880 entries, 0 to 3879
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   code             3880 non-null   object
 1   region           3880 non-null   object
 2   region_ascii     3880 non-null   object
 3   geonameid_admin  3880 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 121.4+ KB


Loading the dataset with alternate names for cities as `alternate_names`:

In [ ]:
alternate_names = pd.read_csv(
    'C:/Users/ASUS/Documents/Projects/Geonames_matching/alternateNamesV2.txt',
    delimiter='\t',
    header=None,
    dtype={'from': object, 'to': object},   # setting the columns datatype
    names=[                                 # naming the columns
        'alternatenameid',                  # alternate name id
        'geonameid',                        # geoname id
        'iso_language',                     # ISO 639 language code
        'alternate_name',                   # alternate city name
        'is_preferred_name',                # an official/preferred name
        'is_short_name',                    # a short name
        'is_colloquial',                    # a colloquial or slang term
        'is_historic',                      # the name was used in the past
        'from',                             # from period when the name was used
        'to'                                # to period when the name was used
    ]
)
alternate_names.head()

,alternatenameid,geonameid,iso_language,alternate_name,is_preferred_name,is_short_name,is_colloquial,is_historic,from,to
0,1284819,2994701,NaN,Roc Mélé,NaN,NaN,NaN,NaN,NaN,NaN
1,1284820,2994701,NaN,Roc Meler,NaN,NaN,NaN,NaN,NaN,NaN
2,4285256,3007683,NaN,Pic des Langounelles,NaN,NaN,NaN,NaN,NaN,NaN
3,1291197,3017832,NaN,Pic de les Abelletes,NaN,NaN,NaN,NaN,NaN,NaN
4,4290387,3017832,NaN,Pic de la Font-Nègre,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
alternate_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16035922 entries, 0 to 16035921
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   alternatenameid    int64  
 1   geonameid          int64  
 2   iso_language       object 
 3   alternate_name     object 
 4   is_preferred_name  float64
 5   is_short_name      float64
 6   is_colloquial      float64
 7   is_historic        float64
 8   from               object 
 9   to                 object 
dtypes: float64(4), int64(2), object(4)
memory usage: 1.2+ GB


Loading the dataset with cities names as `cities`:

In [ ]:
cities = pd.read_csv(
    'C:/Users/ASUS/Documents/Projects/Geonames_matching/cities15000.txt',
    delimiter='\t',
    header=None,
    names=[                    # naming the columns
        'geonameid',           # geoname id
        'name',                # city name
        'name_ascii',          # city ASCII name
        'alternate_names',     # alternate city names
        'latitude',           
        'longitude',
        'feature_class',       
        'feature_code',
        'ISO',                 # ISO 3166 country code
        'cc2',                 # alternate country code
        'admin1 code',         # fipscode
        'admin2 code',         # code for the second level administrative division
        'admin3 code',         # code for the third level administrative division
        'admin4 code',         # code for the fourth level administrative division
        'population',
        'elevation',
        'dem',                 # digital elevation model
        'timezone',            # IANA time zone
        'modification_date'    # date of last modification
    ]
)
cities.head()

,geonameid,name,name_ascii,alternate_names,latitude,longitude,feature_class,feature_code,ISO,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2020-03-03
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,62747,NaN,2,Asia/Dubai,2019-10-24
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,351943,NaN,2,Asia/Dubai,2019-09-09
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,NaN,01,103,NaN,NaN,63482,NaN,118,Asia/Dubai,2019-10-24


In [ ]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27167 entries, 0 to 27166
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   geonameid          27167 non-null  int64  
 1   name               27167 non-null  object 
 2   name_ascii         27167 non-null  object 
 3   alternate_names    24838 non-null  object 
 4   latitude           27167 non-null  float64
 5   longitude          27167 non-null  float64
 6   feature_class      27167 non-null  object 
 7   feature_code       27167 non-null  object 
 8   ISO                27153 non-null  object 
 9   cc2                13 non-null     object 
 10  admin1 code        27159 non-null  object 
 11  admin2 code        22094 non-null  object 
 12  admin3 code        8510 non-null   object 
 13  admin4 code        2628 non-null   object 
 14  population         27167 non-null  int64  
 15  elevation          4256 non-null   float64
 16  dem                271

Creating a `code` column by joining `ISO` and `admin1 code` in the same format as that of the `admin_divisions` dataset for further merge:

In [ ]:
cities['code'] = cities['ISO'] + '.' + cities['admin1 code']
cities.head()

,geonameid,name,name_ascii,alternate_names,latitude,longitude,feature_class,feature_code,ISO,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date,code
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2020-03-03,AD.07
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,62747,NaN,2,Asia/Dubai,2019-10-24,AE.07
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,351943,NaN,2,Asia/Dubai,2019-09-09,AE.05
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,NaN,01,103,NaN,NaN,63482,NaN,118,Asia/Dubai,2019-10-24,AE.01


Getting info about the updated dataset:

In [ ]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27167 entries, 0 to 27166
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   geonameid          27167 non-null  int64  
 1   name               27167 non-null  object 
 2   name_ascii         27167 non-null  object 
 3   alternate_names    24838 non-null  object 
 4   latitude           27167 non-null  float64
 5   longitude          27167 non-null  float64
 6   feature_class      27167 non-null  object 
 7   feature_code       27167 non-null  object 
 8   ISO                27153 non-null  object 
 9   cc2                13 non-null     object 
 10  admin1 code        27159 non-null  object 
 11  admin2 code        22094 non-null  object 
 12  admin3 code        8510 non-null   object 
 13  admin4 code        2628 non-null   object 
 14  population         27167 non-null  int64  
 15  elevation          4256 non-null   float64
 16  dem                271

Loading the dataset with countries info as `countries`:

In [ ]:
countries = pd.read_csv(
    'C:/Users/ASUS/Documents/Projects/Geonames_matching/countryInfo.txt',
    delimiter='\t'
    )
countries.head()

,ISO,ISO3,ISO-Numeric,fips,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",NaN
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NaN,.ag,XCD,Dollar,+1-268,NaN,NaN,en-AG,3576396,NaN,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN,.ai,XCD,Dollar,+1-264,NaN,NaN,en-AI,3573511,NaN,NaN


In [ ]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ISO                 251 non-null    object 
 1   ISO3                252 non-null    object 
 2   ISO-Numeric         252 non-null    int64  
 3   fips                249 non-null    object 
 4   Country             252 non-null    object 
 5   Capital             246 non-null    object 
 6   Area(in sq km)      252 non-null    float64
 7   Population          252 non-null    int64  
 8   Continent           210 non-null    object 
 9   tld                 251 non-null    object 
 10  CurrencyCode        251 non-null    object 
 11  CurrencyName        251 non-null    object 
 12  Phone               247 non-null    object 
 13  Postal Code Format  162 non-null    object 
 14  Postal Code Regex   162 non-null    object 
 15  Languages           249 non-null    object 
 16  geonamei

Renaming the `geonameid` column, so as not to confuse it with the column of the same name in the `cities` dataset:

In [ ]:
countries.rename(columns = {'geonameid':'geonameid_country'}, inplace = True)

Getting info about the updated dataset:

In [ ]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ISO                 251 non-null    object 
 1   ISO3                252 non-null    object 
 2   ISO-Numeric         252 non-null    int64  
 3   fips                249 non-null    object 
 4   Country             252 non-null    object 
 5   Capital             246 non-null    object 
 6   Area(in sq km)      252 non-null    float64
 7   Population          252 non-null    int64  
 8   Continent           210 non-null    object 
 9   tld                 251 non-null    object 
 10  CurrencyCode        251 non-null    object 
 11  CurrencyName        251 non-null    object 
 12  Phone               247 non-null    object 
 13  Postal Code Format  162 non-null    object 
 14  Postal Code Regex   162 non-null    object 
 15  Languages           249 non-null    object 
 16  geonamei

### Creating a working dataset

Merging the `alternate_names` and `cities` datasets on the `geonameid` column and saving as `df_1`:

In [ ]:
df_1 = pd.merge(alternate_names, cities, on="geonameid")
df_1.head()

,alternatenameid,geonameid,iso_language,alternate_name,is_preferred_name,is_short_name,is_colloquial,is_historic,from,to,...,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date,code
0,1297907,3040051,ca,Les Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08
1,1297908,3040051,ca,Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08
2,1904145,3040051,fr,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08
3,1904146,3040051,pl,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08
4,1904147,3040051,es,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08


In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341198 entries, 0 to 341197
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   alternatenameid    341198 non-null  int64  
 1   geonameid          341198 non-null  int64  
 2   iso_language       289221 non-null  object 
 3   alternate_name     341198 non-null  object 
 4   is_preferred_name  8214 non-null    float64
 5   is_short_name      655 non-null     float64
 6   is_colloquial      302 non-null     float64
 7   is_historic        2096 non-null    float64
 8   from               309 non-null     object 
 9   to                 286 non-null     object 
 10  name               341198 non-null  object 
 11  name_ascii         341198 non-null  object 
 12  alternate_names    338040 non-null  object 
 13  latitude           341198 non-null  float64
 14  longitude          341198 non-null  float64
 15  feature_class      341198 non-null  object 
 16  fe

Merging the received dataset `df_1` with the `admin_divisions` dataset on the `code` column and saving as `df_2`:

In [ ]:
df_2 = pd.merge(df_1, admin_divisions, on="code")
df_2.head()

,alternatenameid,geonameid,iso_language,alternate_name,is_preferred_name,is_short_name,is_colloquial,is_historic,from,to,...,admin4 code,population,elevation,dem,timezone,modification_date,code,region,region_ascii,geonameid_admin
0,1297907,3040051,ca,Les Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529
1,1297908,3040051,ca,Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529
2,1904145,3040051,fr,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529
3,1904146,3040051,pl,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529
4,1904147,3040051,es,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529


In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340031 entries, 0 to 340030
Data columns (total 32 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   alternatenameid    340031 non-null  int64  
 1   geonameid          340031 non-null  int64  
 2   iso_language       288127 non-null  object 
 3   alternate_name     340031 non-null  object 
 4   is_preferred_name  8176 non-null    float64
 5   is_short_name      653 non-null     float64
 6   is_colloquial      291 non-null     float64
 7   is_historic        2089 non-null    float64
 8   from               309 non-null     object 
 9   to                 286 non-null     object 
 10  name               340031 non-null  object 
 11  name_ascii         340031 non-null  object 
 12  alternate_names    336874 non-null  object 
 13  latitude           340031 non-null  float64
 14  longitude          340031 non-null  float64
 15  feature_class      340031 non-null  object 
 16  fe

Merging the received dataset `df_2` with the `countries` dataset on the `ISO` column and saving as `df_full`:

In [ ]:
df_full = pd.merge(df_2, countries, on="ISO")
df_full.head()

,alternatenameid,geonameid,iso_language,alternate_name,is_preferred_name,is_short_name,is_colloquial,is_historic,from,to,...,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid_country,neighbours,EquivalentFipsCode
0,1297907,3040051,ca,Les Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,1297908,3040051,ca,Escaldes,NaN,NaN,NaN,NaN,NaN,NaN,...,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
2,1904145,3040051,fr,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
3,1904146,3040051,pl,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
4,1904147,3040051,es,Escaldes-Engordany,NaN,NaN,NaN,NaN,NaN,NaN,...,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN


In [ ]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340031 entries, 0 to 340030
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   alternatenameid     340031 non-null  int64  
 1   geonameid           340031 non-null  int64  
 2   iso_language        288127 non-null  object 
 3   alternate_name      340031 non-null  object 
 4   is_preferred_name   8176 non-null    float64
 5   is_short_name       653 non-null     float64
 6   is_colloquial       291 non-null     float64
 7   is_historic         2089 non-null    float64
 8   from                309 non-null     object 
 9   to                  286 non-null     object 
 10  name                340031 non-null  object 
 11  name_ascii          340031 non-null  object 
 12  alternate_names     336874 non-null  object 
 13  latitude            340031 non-null  float64
 14  longitude           340031 non-null  float64
 15  feature_class       340031 non-nul

Now we have a dataset that includes all the columns required for further work.

Filtering the required countries as a new dataset `df` and putting it on screen:

In [ ]:
df = df_full[(df_full['ISO'] == 'RU') | (df_full['ISO'] == 'BY') | \
             (df_full['ISO'] == 'KG') | (df_full['ISO'] == 'KZ') | \
             (df_full['ISO'] == 'AM') | (df_full['ISO'] == 'GE') | \
             (df_full['ISO'] == 'RS')]
df.head()

,alternatenameid,geonameid,iso_language,alternate_name,is_preferred_name,is_short_name,is_colloquial,is_historic,from,to,...,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid_country,neighbours,EquivalentFipsCode
2160,135616,174875,NaN,Qafan,NaN,NaN,NaN,NaN,NaN,NaN,...,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN
2161,1925363,174875,es,Kapan,NaN,NaN,NaN,NaN,NaN,NaN,...,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN
2162,1925364,174875,en,Kapan,NaN,NaN,NaN,NaN,NaN,NaN,...,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN
2163,1925365,174875,de,Kapan,NaN,NaN,NaN,NaN,NaN,NaN,...,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN
2164,1925366,174875,fa,کاپان,NaN,NaN,NaN,NaN,NaN,NaN,...,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN


Resetting the index:

In [ ]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24848 entries, 0 to 24847
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   alternatenameid     24848 non-null  int64  
 1   geonameid           24848 non-null  int64  
 2   iso_language        21118 non-null  object 
 3   alternate_name      24848 non-null  object 
 4   is_preferred_name   1310 non-null   float64
 5   is_short_name       31 non-null     float64
 6   is_colloquial       24 non-null     float64
 7   is_historic         359 non-null    float64
 8   from                215 non-null    object 
 9   to                  144 non-null    object 
 10  name                24848 non-null  object 
 11  name_ascii          24848 non-null  object 
 12  alternate_names     24844 non-null  object 
 13  latitude            24848 non-null  float64
 14  longitude           24848 non-null  float64
 15  feature_class       24848 non-null  object 
 16  feat

Exporting the new dataset to PostgreSQL as `cities_extended`:

In [ ]:
# df.to_sql('cities_extended', con=engine)

Importing the required columns from PostgreSQL and creating a working dataset as `corpus`:

In [4]:
# we will use all alternate cities names to better train the model:
query = '''SELECT geonameid,
                  alternate_name AS name,
                  region,
                  country,
                  population
           FROM cities_extended;
'''

corpus = pd.read_sql_query(query, con=engine)
corpus.head()

,geonameid,name,region,country,population
0,174875,Qafan,Syunik,Armenia,33160
1,174875,Kapan,Syunik,Armenia,33160
2,174875,Kapan,Syunik,Armenia,33160
3,174875,Kapan,Syunik,Armenia,33160
4,174875,کاپان,Syunik,Armenia,33160


In [5]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24848 entries, 0 to 24847
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   geonameid   24848 non-null  int64 
 1   name        24848 non-null  object
 2   region      24848 non-null  object
 3   country     24848 non-null  object
 4   population  24848 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 970.8+ KB


### Preprocessing the working dataset

Writing a function to convert all cities names in different languages to unicode:

In [7]:
# The function recieves a "raw" word and returns it converted:
def prepare_word(word):
    
# The `unicode` module converts any language into Latin:
    clear_word = unidecode.unidecode(word)

    return clear_word

# Checking how the function works. The list is especially chosen to include 
# alternate names in different languages:
[print (prepare_word(i))for i in ['Кариба','كاريبا ','کاریبا، زمبابوے','卡里巴','카리바']]
;

Kariba
kryb 
khryb, zmbbwy
Qia Li Ba 
kariba


''

The function works fine. Applying it to the `name` column of the `corpus`:

In [8]:
corpus['name'] = corpus['name'].apply(prepare_word)
corpus.head(10)

,geonameid,name,region,country,population
0,174875,Qafan,Syunik,Armenia,33160
1,174875,Kapan,Syunik,Armenia,33160
2,174875,Kapan,Syunik,Armenia,33160
3,174875,Kapan,Syunik,Armenia,33160
4,174875,khpn,Syunik,Armenia,33160
5,174875,Kapan,Syunik,Armenia,33160
6,174875,Kapan,Syunik,Armenia,33160
7,174875,https://en.wikipedia.org/wiki/Kapan,Syunik,Armenia,33160
8,174875,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...,Syunik,Armenia,33160
9,174875,Kapan,Syunik,Armenia,33160


Deleting all rows with internet links and special symbols from the dataset:

In [9]:
corpus = corpus[~corpus['name'].str.contains("%")]
corpus = corpus[~corpus['name'].str.contains("https://")]
corpus = corpus[~corpus['name'].str.contains("@")]
corpus.head(10)

,geonameid,name,region,country,population
0,174875,Qafan,Syunik,Armenia,33160
1,174875,Kapan,Syunik,Armenia,33160
2,174875,Kapan,Syunik,Armenia,33160
3,174875,Kapan,Syunik,Armenia,33160
4,174875,khpn,Syunik,Armenia,33160
5,174875,Kapan,Syunik,Armenia,33160
6,174875,Kapan,Syunik,Armenia,33160
9,174875,Kapan,Syunik,Armenia,33160
10,174875,Kapan,Syunik,Armenia,33160
11,174875,kapan,Syunik,Armenia,33160


### **Summary:**

The datasets `admin_divisions`, `alternate_names`, `cities` and `countries` are loaded. In `cities`, a new `code` column is created by joining  the `ISO` and `admin1 code` columns row by row. In `countries`, the `geonameid` column is renamed as `geonameid_country`. 

Four datasets are merged into one full dataset, which is then filtered for the required countries and exported into PostgreSQL as `cities_extended`. From  it, the working dataset with `geonameid`, `region`, `country`, `population` and `alternate_name` (renamed as `name`) columns are imported as `corpus`.

A function `prepare_word` for translating cities names from any language into Latin is written, tested and then applied to the `name` column. The `name` column of the `corpus` is also cleared from any special symbols and internet links. 

The `corpus` dataset is ready for further use.

## Applying the Sentence Transformer

Dropping duplicates from, applying the `values` method to the `name` column and saving as `names`:

In [10]:
names = corpus.name.drop_duplicates().values
names[-10:]

array(['andyr', 'anadeiri', 'Anadyris', 'Anadira', 'anaadir',
       'A Na De Er ', 'DYR', 'kaciirciin', 'Qagiirgiin', 'RUDYR'],
      dtype=object)

Loading LaBSE sentence transformer as `labse`:

In [11]:
labse = SentenceTransformer('sentence-transformers/LaBSE')

Creating embeddings from `names` using LaBSE:

In [12]:
embeddings = labse.encode(names)
embeddings.shape

(13227, 768)

Applying the semantic search to a random city name and saving the results as a new dataset `result`:

In [13]:
# Running a search for the 'Санкт-Петербург' query as one of the more complex ones
# Top_k limits the search results to a hundred
result = pd.DataFrame(util.semantic_search(labse.encode('Санкт-Петербург'), embeddings, top_k=100)[0])
result = result.assign(name=names[result.corpus_id])

# Putting top ten results on screen:
result.head(10)

,corpus_id,score,name
0,3033,0.959648,Sankt-Petersburg
1,3029,0.957673,Sankt Petersburg
2,3098,0.950107,Sankt-Peterburq
3,3030,0.946336,Saint Petersburg
4,3051,0.943733,St. Petersburg
5,3040,0.942912,St Petersburg
6,3037,0.934982,Sant Petersburg
7,3034,0.931404,Sant Petersburgo
8,3044,0.928642,Sankt-Peterburgo
9,3087,0.925170,St.-Petersburg


**Summary:**

The `name` column of the `corpus` is cleared from duplicates, converted using `values` method and saved as `names`. From them, embeddings are created using LaBSE sentence transformer. Semantic search for the 'Санкт-Петербург' query demonstrates a suitable match ('Sankt-Petersburg') in the first row with the score of 0.96 and an optimal match ('Saint Petersburg') in the fourth row with the score of 0.95. 

So, the chosen algorithm will be appropriate for the needs of the project.

## Expanding the resulting dataset

Merging the `result` and `corpus` datasets on the `name` to add the required columns and dropping duplicates from the results:

In [14]:
# Merging the `result` and `corpus` datasets on the `name`:
result = pd.merge(result, corpus, on="name")

# Dropping duplicates from the `result` dataset:
result = result.drop_duplicates()

# Dropping rows with duplicates in the `geonameid` column
# to have only unique locations in the results:
result = result.drop_duplicates(subset=['geonameid'])

# Sorting the results, so, in case of there being more than one location with 
# the same name, the location with the bigger population would be recommended first:
result = result.sort_values(by = ["score", "population"], ascending = False)

# Saving the top ten results:
result = result.head(10)

# Putting the resulting dataset on screen:
result

,corpus_id,score,name,geonameid,region,country,population
0,3033,0.959648,Sankt-Petersburg,498817,St.-Petersburg,Russia,5351935
77,8718,0.646674,Svetagorsk,485639,Leningradskaya Oblast',Russia,15733
78,2031,0.636810,Petropavlovsk,1520172,North Kazakhstan,Kazakhstan,200920
85,2031,0.636810,Petropavlovsk,2122104,Kamchatka,Russia,181216
84,2031,0.636810,Petropavlovsk,496275,Sverdlovsk Oblast,Russia,33500
86,10350,0.634541,Petrozabontsk,509820,Karelia,Russia,279190
87,5602,0.630765,Petrowskoje,485698,Stavropol Kray,Russia,39727
89,3212,0.628174,Petrogradka,8504948,St.-Petersburg,Russia,130455
97,10556,0.623883,Jekaterinburgas,1486209,Sverdlovsk Oblast,Russia,1495066
98,2199,0.622277,Petrovgrad,783814,Vojvodina,Serbia,79773


Dropping the unnecessary columns from the resulting dataset:

In [15]:
result = result.drop(['corpus_id', 'population'], axis=1)
result

,score,name,geonameid,region,country
0,0.959648,Sankt-Petersburg,498817,St.-Petersburg,Russia
77,0.646674,Svetagorsk,485639,Leningradskaya Oblast',Russia
78,0.636810,Petropavlovsk,1520172,North Kazakhstan,Kazakhstan
85,0.636810,Petropavlovsk,2122104,Kamchatka,Russia
84,0.636810,Petropavlovsk,496275,Sverdlovsk Oblast,Russia
86,0.634541,Petrozabontsk,509820,Karelia,Russia
87,0.630765,Petrowskoje,485698,Stavropol Kray,Russia
89,0.628174,Petrogradka,8504948,St.-Petersburg,Russia
97,0.623883,Jekaterinburgas,1486209,Sverdlovsk Oblast,Russia
98,0.622277,Petrovgrad,783814,Vojvodina,Serbia


Converting the resulting dataset to a list of tuples:

In [16]:
result.to_dict(orient='records')

[{'score': 0.9596481323242188,
  'name': 'Sankt-Petersburg',
  'geonameid': 498817,
  'region': 'St.-Petersburg',
  'country': 'Russia'},
 {'score': 0.6466743350028992,
  'name': 'Svetagorsk',
  'geonameid': 485639,
  'region': "Leningradskaya Oblast'",
  'country': 'Russia'},
 {'score': 0.6368100643157959,
  'name': 'Petropavlovsk',
  'geonameid': 1520172,
  'region': 'North Kazakhstan',
  'country': 'Kazakhstan'},
 {'score': 0.6368100643157959,
  'name': 'Petropavlovsk',
  'geonameid': 2122104,
  'region': 'Kamchatka',
  'country': 'Russia'},
 {'score': 0.6368100643157959,
  'name': 'Petropavlovsk',
  'geonameid': 496275,
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia'},
 {'score': 0.6345407962799072,
  'name': 'Petrozabontsk',
  'geonameid': 509820,
  'region': 'Karelia',
  'country': 'Russia'},
 {'score': 0.6307648420333862,
  'name': 'Petrowskoje',
  'geonameid': 485698,
  'region': 'Stavropol Kray',
  'country': 'Russia'},
 {'score': 0.6281737089157104,
  'name': 'Petrogra

### Creating a function for geoname queries

Writing the `geoname` function. The function receives a city name as a query and returns a list of tuples with five recommended names and a geonameid, a region, a country and a cosine similarity for each city name: 

In [17]:
def geoname(query):
    result = pd.DataFrame(util.semantic_search(labse.encode(query), embeddings, top_k=100)[0])
    result = result.assign(name=names[result.corpus_id])
    result = pd.merge(result, corpus, on="name")
    result = result.drop_duplicates()
    result = result.drop_duplicates(subset=['geonameid'])
    result = result.sort_values(by = ["score", "population"], ascending = False)
    result = result.head(5)   # saving top 5 rows in the resulting dataset
    result = result.drop(['corpus_id', 'population'], axis=1)
    result = result.to_dict(orient='records')
    
    return result

Testing the `geoname` function:

In [18]:
geoname('Киров')

[{'score': 0.9048265218734741,
  'name': 'Kirov',
  'geonameid': 548408,
  'region': 'Kirov Oblast',
  'country': 'Russia'},
 {'score': 0.9048265218734741,
  'name': 'Kirov',
  'geonameid': 548410,
  'region': 'Kaluga Oblast',
  'country': 'Russia'},
 {'score': 0.8215348720550537,
  'name': 'Kirovsk',
  'geonameid': 548391,
  'region': 'Murmansk',
  'country': 'Russia'},
 {'score': 0.8215348720550537,
  'name': 'Kirovsk',
  'geonameid': 548392,
  'region': "Leningradskaya Oblast'",
  'country': 'Russia'},
 {'score': 0.8159705400466919,
  'name': 'Kirovgrad',
  'geonameid': 1503335,
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia'}]

**Summary:**

To receive all the required columns, the `result` dataset is merged with the `corpus` on `name` and cleared from fully duplicated rows and rows with duplicates in the `geonameid` column. Then, the dataset is sorted, so, in case of there being more than one location with the same name, the location with the bigger population would be recommended first. No longer necessary `corpus_id` and `population` columns are dropped and the `result` dataset is converted into the list of tuples as per requirement.

The `geoname` function is created and tested for optimizing the further use of the solution.

## Testing the solution

Loading the test dataset:

In [20]:
geo_test = pd.read_csv(
    'C:/Users/ASUS/Documents/Projects/Geonames_matching/geo_test.csv',
    sep='\;',
    engine='python'
    )
geo_test.head()

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan
3,Москва,Moscow,Moscow,Russia
4,Алматы,Almaty,Almaty,Kazakhstan


Getting five random queries for further testing:

In [ ]:
geo_test.sample(n=5)

,query,name,region,country
160,Каспийск,Kaspiysk,Dagestan,Russia
238,Ахалцихе,Akhaltsikhe,Samtskhe-Javakheti,Georgia
70,Павловск,Pavlovsk,St.-Petersburg,Russia
130,Уссурийск,Ussuriysk,Primorye,Russia
220,Новоалтайск,Novoaltaysk,Altai Krai,Russia


Testing the queries:

In [21]:
geoname('Каспийск')

[{'score': 0.9185491800308228,
  'name': 'Kaspijsk',
  'geonameid': 551847,
  'region': 'Dagestan',
  'country': 'Russia'},
 {'score': 0.7586101293563843,
  'name': 'Capajevsk',
  'geonameid': 569955,
  'region': 'Samara Oblast',
  'country': 'Russia'},
 {'score': 0.7252591848373413,
  'name': 'Kopejsk',
  'geonameid': 1502603,
  'region': 'Chelyabinsk',
  'country': 'Russia'},
 {'score': 0.6967259049415588,
  'name': "K'apshag'ai",
  'geonameid': 1519948,
  'region': 'Almaty Oblysy',
  'country': 'Kazakhstan'},
 {'score': 0.6966315507888794,
  'name': 'Kabakovsk',
  'geonameid': 1492663,
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia'}]

In [22]:
geoname('Ахалцихе')

[{'score': 0.8862636089324951,
  'name': 'Achalciche',
  'geonameid': 615860,
  'region': 'Samtskhe-Javakheti',
  'country': 'Georgia'},
 {'score': 0.7279251217842102,
  'name': 'Akhalsenaki',
  'geonameid': 612053,
  'region': 'Samegrelo and Zemo Svaneti',
  'country': 'Georgia'},
 {'score': 0.7128996253013611,
  'name': 'Alacahisar',
  'geonameid': 788975,
  'region': 'Central Serbia',
  'country': 'Serbia'},
 {'score': 0.7044004797935486,
  'name': 'Ashcharak',
  'geonameid': 616877,
  'region': 'Aragatsotn',
  'country': 'Armenia'},
 {'score': 0.7003030776977539,
  'name': 'Kalachly',
  'geonameid': 1505074,
  'region': 'Omsk Oblast',
  'country': 'Russia'}]

In [23]:
geoname('Павловск')

[{'score': 0.9115622043609619,
  'name': 'Pavlovszk',
  'geonameid': 512052,
  'region': 'St.-Petersburg',
  'country': 'Russia'},
 {'score': 0.9028421640396118,
  'name': 'Pavlovsk',
  'geonameid': 512053,
  'region': 'Voronezh Oblast',
  'country': 'Russia'},
 {'score': 0.8551024794578552,
  'name': 'Pavlovskaya',
  'geonameid': 512051,
  'region': 'Krasnodar Krai',
  'country': 'Russia'},
 {'score': 0.8200539350509644,
  'name': 'Polevskoy',
  'geonameid': 1494573,
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia'},
 {'score': 0.8198615312576294,
  'name': 'Pallasovk',
  'geonameid': 513042,
  'region': 'Volgograd Oblast',
  'country': 'Russia'}]

In [24]:
geoname('Уссурийск')

[{'score': 0.9266659021377563,
  'name': 'Ussuryjsk',
  'geonameid': 2014006,
  'region': 'Primorye',
  'country': 'Russia'},
 {'score': 0.7966721057891846,
  'name': 'Ussuri',
  'geonameid': 2020812,
  'region': 'Primorye',
  'country': 'Russia'},
 {'score': 0.769874095916748,
  'name': 'Ussinsk',
  'geonameid': 863061,
  'region': 'Komi',
  'country': 'Russia'},
 {'score': 0.7322225570678711,
  'name': 'Uryupinsk',
  'geonameid': 478544,
  'region': 'Volgograd Oblast',
  'country': 'Russia'},
 {'score': 0.7229852676391602,
  'name': 'Ozersk',
  'geonameid': 1538634,
  'region': 'Chelyabinsk',
  'country': 'Russia'}]

In [25]:
geoname('Новоалтайск')

[{'score': 0.9462507963180542,
  'name': 'Novoaltaysk',
  'geonameid': 1497173,
  'region': 'Altai Krai',
  'country': 'Russia'},
 {'score': 0.8325079679489136,
  'name': 'Novonikolayevsk',
  'geonameid': 1496747,
  'region': 'Novosibirsk Oblast',
  'country': 'Russia'},
 {'score': 0.8174968957901001,
  'name': 'Novosahtinsk',
  'geonameid': 517963,
  'region': 'Rostov',
  'country': 'Russia'},
 {'score': 0.801964282989502,
  'name': "Novoul'yanovsk",
  'geonameid': 517766,
  'region': 'Ulyanovsk',
  'country': 'Russia'},
 {'score': 0.7982013821601868,
  'name': 'Novotroytsk',
  'geonameid': 517836,
  'region': 'Orenburg Oblast',
  'country': 'Russia'}]

**Summary:**

The test dataset is loaded. Five random queries are chosen from it and tested with the `geonames` function. For all five queries ("Каспийск", "Ахалцихе", "Павловск", "Уссурийск", "Новоалтайск") the solution finds a location with the correct geonameid in the first row. In terms of spelling, "Новоалтайск" is a full match with the recommended geoname (score=0.95); "Каспийск", "Павловск" and "Уссурийск" have a minor difference of one letter each (scores from 0.91 to 0.93); "Ахалцихе" is a bit off (score=0.89), but still fully recognizable. 

The chosen solution works fine and might be used by a human operator for the designated task.

## Conclusion

During the preprocessing the required datasets `admin_divisions`, `alternate_names`, `cities` and `countries` are loaded. In `cities`, a new `code` column is created. In `countries`, the `geonameid` column is renamed as `geonameid_country`. Four datasets are merged into one full dataset, which is then filtered for the required countries and exported into PostgreSQL as `cities_extended`. From  it, the working dataset with `geonameid`, `region`, `country`, `population` and `alternate_name` (renamed as `name`) columns are imported as `corpus`. A function `prepare_word` for translating cities names from any language into Latin is written, tested and then applied to the `name` column. The `name` column of the `corpus` is also cleared from any special symbols and internet links. 

The `name` column of the `corpus` is cleared from duplicates, converted using `values` method and saved as `names`. From them, embeddings are created using LaBSE sentence transformer. Semantic search for the 'Санкт-Петербург' query demonstrates a suitable match ('Sankt-Petersburg') in the first row with the score of 0.96 and an optimal match ('Saint Petersburg') in the fourth row with the score of 0.95.

To receive all the required columns, the `result` dataset is merged with the `corpus` on `name` and cleared from fully duplicated rows and rows with duplicates in the `geonameid` column. Then, the dataset is sorted, so, in case of there being more than one location with the same name, the location with the bigger population would be recommended first. No longer necessary `corpus_id` and `population` columns are dropped and the `result` dataset is converted into the list of tuples as per requirement. The `geoname` function is created and tested for optimizing the further use of the solution.

The test dataset is loaded. Five random queries are chosen from it and tested with the `geonames` function. For all five queries the solution finds a location with the correct geonameid in the first row. In terms of spelling, "Новоалтайск" is a full match with the recommended geoname (score=0.95); "Каспийск", "Павловск" and "Уссурийск" have a minor difference of one letter each (scores from 0.91 to 0.93); "Ахалцихе" is a bit off (score=0.89), but still fully recognizable. 

Considering that the algorithm will be used as a recommendation system for a human operator, the Sentence Transformer based solution works quite well and might be used for the designated task.